<a href="https://colab.research.google.com/github/chihiro1412/Visual-Sentiment-Subteam-CNN/blob/main/Web_Application_MDP_Flash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install packages**

In [1]:
! pip install -q git+https://github.com/huggingface/transformers datasets
! pip install -U accelerate
! pip install -U transformers
! pip install streamlit
! pip install pyngrok
! npm install localtunnel

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.8/731.8 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created w

Mount at Google drive.  
To ensure web app could run normally, need to have saved model in the following directory. You could also change code in web app to change directory.  
current specified directory: content/drive/MyDrive/saved_models/fer_vit_model

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Run code for web app**

In [23]:
%%writefile app.py
import streamlit as st
from datasets import *
from transformers import ViTFeatureExtractor
from transformers import ViTModel
from transformers import TrainingArguments, Trainer
from transformers.modeling_outputs import SequenceClassifierOutput
import numpy as np
import pandas as pd
import torch.nn as nn
import torch
import pickle
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import accelerate
import transformers


# helper methods and classes
bin_string_labels = ["Negative", "Positive"]
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')


class ViTForImageClassification(nn.Module):
    #define architecture
    def __init__(self, num_labels=len(bin_string_labels)):
        super(ViTForImageClassification, self).__init__()
        self.vit = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.vit.config.hidden_size, num_labels)
        self.num_labels = num_labels

    #define a forward pass through that architecture + loss computation
    def forward(self, pixel_values, labels):
        outputs = self.vit(pixel_values=pixel_values)
        output = self.dropout(outputs.last_hidden_state[:,0])
        logits = self.classifier(output)

        loss = None
        if labels is not None:
          loss_fct = nn.CrossEntropyLoss()
          loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

def grayscale_to_rgb_2d(grayscale_matrix):
    """
    Convert a 2D matrix of grayscale values (0-255) to RGB format.

    Args:
    - grayscale_matrix (list of list of int): A 2D matrix of grayscale values.

    Returns:
    - list of list of tuple: A 2D matrix of RGB triplets.
    """
    return [[[value, value, value] for value in row] for row in grayscale_matrix]

def data_transformation(image):
  # Convert to grayscale
  image = image.convert('L')

  # Resize to 48*48
  image = image.resize((48, 48))

  # Convert to pixel data
  pixel_data = np.array(image)

  result = grayscale_to_rgb_2d(pixel_data)

  return result

def preprocess_images(examples):
    """ Prepare datasets for vision transformer
    input: dataset with images in their orignal size
    output: dataset with pixelvalues computed by the feature extractor added """
    # get batch of images
    images = examples['img']
    # convert to list of NumPy arrays of shape (C, H, W)
    images = [np.array(image, dtype=np.uint8) for image in images]
    images = [np.moveaxis(image, source=-1, destination=0) for image in images]
    # preprocess and add pixel_values
    inputs = feature_extractor(images=images)
    examples['pixel_values'] = inputs['pixel_values']

    return examples


@st.cache_data
def load_model():
  model = ViTForImageClassification()
  model.load_state_dict(torch.load('/content/drive/MyDrive/saved_models/fer_vit_model',map_location=torch.device('cpu')))
  return model

with st.spinner('Model is being loaded..'):
  model=load_model()

st.write("""
         # Image Sentiment Classification 	:computer:
         """
         )
file = st.file_uploader("Please upload an brain scan file", type=["jpg", "png"])


import cv2
from PIL import Image, ImageOps
import numpy as np
st.set_option('deprecation.showfileUploaderEncoding', False)
def import_and_predict(test_image, model):
       img = data_transformation(test_image)
       Image.fromarray(np.array(img), 'RGB')
       data = {
            'img': [img],
            'label': -1
        }

       df = pd.DataFrame(data)
       bin_test_ds = Dataset.from_pandas(df)


       features = Features({
    'label': ClassLabel(names=['Negative', 'Positive']),
    'img': Array3D(dtype="int64", shape=(3,48,48)),
    'pixel_values': Array3D(dtype="float32", shape=(3, 224, 224)),
})

       preprocessed_bin_test_ds = bin_test_ds.map(preprocess_images, batched=True, features=features)
       with open('preprocessed_test_ds.pickle', 'wb') as handle:
          pickle.dump(preprocessed_bin_test_ds, handle, protocol=pickle.HIGHEST_PROTOCOL)
       pixel_data = torch.tensor(preprocessed_bin_test_ds[0]["pixel_values"]).unsqueeze(0)
       model.eval()
       pred = model(pixel_data , None)
       prediction = pred.logits.argmax(1).item() # 0 or 1
       return prediction


if file is None:
    st.text("Please upload an image file")
else:
    image = Image.open(file)
    st.image(image, width=300)
    prediction = import_and_predict(image, model)

    if prediction == 0:
      st.write("""
         # Negative Emotion :sob:
         """
         )
    else:
      st.write("""
         # Positive Emotion :smile:
         """
         )

Overwriting app.py


**Run Web App **

First, get the IP for this session. You will need to copy and paste it later.

In [8]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 34.80.253.84




1.   Run the following code block.
2.   Click on the url pops up.
3.   Enter the IP address from code above.





In [24]:
!streamlit run app.py &>/content/logs.txt &
!npx localtunnel --port 8501

npx: installed 22 in 3.247s
your url is: https://mean-forks-poke.loca.lt
^C
